<a href="https://colab.research.google.com/github/aSafarpoor/OSN_FAD/blob/main/CLS/FAD%2BNN%2BNode2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
!pip install node2vec

In [156]:
!pip install torch_geometric

In [172]:
import networkx as nx
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from node2vec import Node2Vec
import random
SEED = 10

In [190]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from sklearn.metrics import roc_curve,accuracy_score, roc_auc_score, confusion_matrix



# read data

In [159]:
edges = []
with open('graph.txt', 'r') as file:
    for line in file:
        # Split each line into two numbers and convert them to integers
        pair = list(map(int, line.strip().split()))
        edges.append(pair)


with open('train.txt', 'r') as file:
    lines = file.readlines()
    btrain = list(map(float, lines[0].strip().split()))
    strain = list(map(float, lines[1].strip().split()))

with open('test.txt', 'r') as file:
    lines = file.readlines()
    btest = list(map(float, lines[0].strip().split()))
    stest = list(map(float, lines[1].strip().split()))

In [160]:
print("Length of edges:", len(edges),
      "| Length of btrain:", len(btrain),
      "| Length of btest:", len(btest),
      "| Length of strain:", len(strain),
      "| Length of stest:", len(stest))

Length of edges: 6818501 | Length of btrain: 20000 | Length of btest: 158377 | Length of strain: 10000 | Length of stest: 81263


# data preparation

In [161]:
bigG = nx.Graph()
bigG.add_edges_from(edges)

In [162]:
trainG = bigG.subgraph(btrain+strain).copy()
testG = bigG.subgraph(btest+stest).copy()

In [163]:
print(len(bigG.nodes()),len(trainG.nodes()),len(testG.nodes()))

269640 30000 239640


In [164]:
def reindex_graph_and_dictionaries(G, train, test):
    new_index_mapping = {old_index: new_index for new_index, old_index in enumerate(G.nodes())}
    G_reindexed = nx.relabel_nodes(G, new_index_mapping)

    train = [new_index_mapping[int(node)] for node in train]
    test =  [new_index_mapping[int(node)] for node in test]

    return G_reindexed, train, test

In [165]:
trainG,btrain,strain =reindex_graph_and_dictionaries(trainG, btrain,strain)
testG,btest,stest =reindex_graph_and_dictionaries(testG, btest,stest)

In [176]:
trainDictionaryX = {}
trainDictionaryY = {}
testDictionaryX = {}
testDictionaryY = {}

random.seed(SEED)

c=0
for node in btrain:
    if random.random() < 0.20:
        c+=1
        trainDictionaryX[node] = [0]
    else:
        trainDictionaryX[node] = [0.5]
    trainDictionaryY[node] = 0
print(c)

c=0
for node in strain:
    if random.random() < 0.20:
        c+=1
        trainDictionaryX[node] = [1]
    else:
        trainDictionaryX[node] = [0.5]
    trainDictionaryY[node] = 1
print(c)

c=0
for node in btest:
    if random.random() < 0.1:
        c+=1
        testDictionaryX[node] = [0]
    else:
        testDictionaryX[node] = [0.5]
    testDictionaryY[node] = 0
print(c)

c=0
for node in stest:
    if random.random() < 0.1:
        c+=1
        testDictionaryX[node] = [1]
    else:
        testDictionaryX[node] = [0.5]
    testDictionaryY[node] = 1
print(c)

4038
1986
15631
8053


In [178]:
# print(len(btrain))
# print(len(strain))
# print(len(btest))
# print(len(stest))


#model

In [179]:
# Prepare data (assuming PyTorch Geometric format for graphs)
def prepare_data(graph, dictionaryX, dictionaryY):
    x = torch.tensor([dictionaryX[node] for node in graph.nodes()], dtype=torch.float)
    y = torch.tensor([dictionaryY[node] for node in graph.nodes()], dtype=torch.long)
    # edge_index = torch.tensor(list(graph.edges())).t().contiguous()
    edge_index = torch.tensor(list(graph.edges()), dtype=torch.long).t().contiguous()
    return x, y, edge_index

In [180]:
def pure_sybil_test(test_x, test_y):
    print(len(test_x))
    mask = (test_y == 1)
    x = test_x[mask]
    y = test_y[mask]
    print(len(x))
    return x, y, mask

In [181]:
train_x, train_y, train_edge_index = prepare_data(trainG, trainDictionaryX, trainDictionaryY)
test_x, test_y, test_edge_index = prepare_data(testG, testDictionaryX, testDictionaryY)
test_x2, test_y2, mask2 = pure_sybil_test(test_x, test_y)


239640
81263


In [205]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=True):
        super(GCN, self).__init__()
        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        self.num_classes = output_dim
        for i in range(num_layers):
            if i == 0:  # First layer
                self.convs.append(GCNConv(input_dim, hidden_dim, bias=False))
            elif i == num_layers - 1:  # Last layer
                self.convs.append(GCNConv(hidden_dim, output_dim, bias=False))
            else:  # Middle layers
                self.convs.append(GCNConv(hidden_dim, hidden_dim, bias=False))

    def forward(self, x, edge_index):
        h = x
        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
            i += 1

        if self.num_classes == 1:
            return F.sigmoid(h), h
        else:
            return F.softmax(h, dim=1)




class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1)  # Single head in the final layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)  # ELU is commonly used in GAT models instead of ReLU
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [204]:
# Initialize model, optimizer, and loss function
input_dim = 1
hidden_dim = 16
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GCN(input_dim, hidden_dim, output_dim, num_layers=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in tqdm(range(400)):
    optimizer.zero_grad()
    out = model(train_x, train_edge_index)
    loss = loss_fn(out, train_y)
    loss.backward()
    optimizer.step()
    if (epoch+1)%20 == 0 or epoch==0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute metrics
    acc = accuracy_score(test_y, pred)
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')  # Adjust depending on binary or multi-class
    cm = confusion_matrix(test_y, pred)
    cm = cm.astype('float') / len(pred)


    print(f'Accuracy: {acc:.3f}')
    print(f'AUC: {auc:.3f}')
    print('Confusion Matrix:')
    print(f"                  Predicted Negative    Predicted Positive")
    print(f"Actual Negative       {cm[0, 0]:.3f}           {cm[0, 1]:.3f}")
    print(f"Actual Positive       {cm[1, 0]:.3f}           {cm[1, 1]:.3f}")

    print("\n--------------------------just sybils-----------------------------\n")

    test_out = model(test_x, test_edge_index)
    test_out = test_out[mask2]
    pred = test_out.argmax(dim=1).cpu()
    acc = accuracy_score(test_y2, pred)
    print(f'Accuracy: {acc:.3f}')


with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute AUC score as usual
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')
    fpr, tpr, thresholds = roc_curve(test_y, test_out[:, 1])
    j_scores = tpr - fpr
    optimal_threshold = thresholds[j_scores.argmax()]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc = accuracy_score(test_y, pred)

    # just sybil
    test_out = test_out[mask2]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc_sybils = accuracy_score(test_y2, pred)
    print("Accuracy with optimal threshold:", acc)
    print("Accuracy with optimal threshold just for sybils:", acc_sybils)

  0%|          | 1/400 [00:00<02:43,  2.44it/s]

Epoch 1, Loss: 0.7187546491622925


  5%|▌         | 21/400 [00:06<01:41,  3.75it/s]

Epoch 20, Loss: 0.6340564489364624


 10%|█         | 41/400 [00:10<00:57,  6.23it/s]

...

 95%|█████████▌| 381/400 [01:09<00:03,  6.23it/s]

Epoch 380, Loss: 0.6282768845558167


100%|██████████| 400/400 [01:13<00:00,  5.48it/s]

Epoch 400, Loss: 0.621724009513855


Accuracy: 0.661
AUC: 0.393
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.661           0.000
Actual Positive       0.339           0.000

--------------------------just sybils-----------------------------

Accuracy: 0.000
Accuracy with optimal threshold: 0.339555166082457
Accuracy with optimal threshold just for sybils: 0.9996308282982415


In [200]:
# Initialize model, optimizer, and loss function
input_dim = 1
hidden_dim = 4
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GCN(input_dim, hidden_dim, output_dim, num_layers=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in tqdm(range(900)):
    optimizer.zero_grad()
    out = model(train_x, train_edge_index)
    loss = loss_fn(out, train_y)
    loss.backward()
    optimizer.step()
    if (epoch+1)%20 == 0 or epoch==0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute metrics
    acc = accuracy_score(test_y, pred)
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')  # Adjust depending on binary or multi-class
    cm = confusion_matrix(test_y, pred)
    cm = cm.astype('float') / len(pred)


    print(f'Accuracy: {acc:.3f}')
    print(f'AUC: {auc:.3f}')
    print('Confusion Matrix:')
    print(f"                  Predicted Negative    Predicted Positive")
    print(f"Actual Negative       {cm[0, 0]:.3f}           {cm[0, 1]:.3f}")
    print(f"Actual Positive       {cm[1, 0]:.3f}           {cm[1, 1]:.3f}")

    print("\n--------------------------just sybils-----------------------------\n")

    test_out = model(test_x, test_edge_index)
    test_out = test_out[mask2]
    pred = test_out.argmax(dim=1).cpu()
    acc = accuracy_score(test_y2, pred)
    print(f'Accuracy: {acc:.3f}')

# model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute AUC score as usual
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')
    fpr, tpr, thresholds = roc_curve(test_y, test_out[:, 1])
    j_scores = tpr - fpr
    optimal_threshold = thresholds[j_scores.argmax()]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc = accuracy_score(test_y, pred)

    # just sybil
    test_out = test_out[mask2]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc_sybils = accuracy_score(test_y2, pred)
    print("Accuracy with optimal threshold:", acc)
    print("Accuracy with optimal threshold just for sybils:", acc_sybils)

  0%|          | 1/900 [00:00<01:57,  7.66it/s]

Epoch 1, Loss: 0.6871949434280396


  2%|▏         | 21/900 [00:02<01:58,  7.42it/s]

Epoch 20, Loss: 0.6314573287963867


  5%|▍         | 41/900 [00:05<01:54,  7.51it/s]

...

 98%|█████████▊| 881/900 [02:14<00:02,  7.43it/s]

Epoch 880, Loss: 0.617091715335846


100%|██████████| 900/900 [02:17<00:00,  6.52it/s]

Epoch 900, Loss: 0.619329035282135


Accuracy: 0.661
AUC: 0.402
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.661           0.000
Actual Positive       0.339           0.000

--------------------------just sybils-----------------------------

Accuracy: 0.000
Accuracy with optimal threshold: 0.3423844099482557
Accuracy with optimal threshold just for sybils: 0.9957422197063854


In [201]:
# Initialize model, optimizer, and loss function
input_dim = 1
hidden_dim = 16
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GAT(input_dim, hidden_dim, output_dim, heads=4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in tqdm(range(400)):
    optimizer.zero_grad()
    out = model(train_x, train_edge_index)
    loss = loss_fn(out, train_y)
    loss.backward()
    optimizer.step()
    if (epoch+1)%20 == 0 or epoch==0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute metrics
    acc = accuracy_score(test_y, pred)
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')  # Adjust depending on binary or multi-class
    cm = confusion_matrix(test_y, pred)
    cm = cm.astype('float') / len(pred)


    print(f'Accuracy: {acc:.3f}')
    print(f'AUC: {auc:.3f}')
    print('Confusion Matrix:')
    print(f"                  Predicted Negative    Predicted Positive")
    print(f"Actual Negative       {cm[0, 0]:.3f}           {cm[0, 1]:.3f}")
    print(f"Actual Positive       {cm[1, 0]:.3f}           {cm[1, 1]:.3f}")


    print("\n--------------------------just sybils-----------------------------\n")

    test_out = model(test_x, test_edge_index)
    test_out = test_out[mask2]
    pred = test_out.argmax(dim=1).cpu()
    acc = accuracy_score(test_y2, pred)
    print(f'Accuracy: {acc:.3f}')

# model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute AUC score as usual
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')
    fpr, tpr, thresholds = roc_curve(test_y, test_out[:, 1])
    j_scores = tpr - fpr
    optimal_threshold = thresholds[j_scores.argmax()]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc = accuracy_score(test_y, pred)

    # just sybil
    test_out = test_out[mask2]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc_sybils = accuracy_score(test_y2, pred)
    print("Accuracy with optimal threshold:", acc)
    print("Accuracy with optimal threshold just for sybils:", acc_sybils)

  0%|          | 1/400 [00:00<03:31,  1.89it/s]

Epoch 1, Loss: 0.7374464869499207


  5%|▌         | 20/400 [00:11<03:53,  1.63it/s]

Epoch 20, Loss: 0.6248778700828552


 10%|█         | 40/400 [00:22<04:01,  1.49it/s]

...

 95%|█████████▌| 380/400 [03:39<00:13,  1.44it/s]

Epoch 380, Loss: 0.397045373916626


100%|██████████| 400/400 [03:50<00:00,  1.74it/s]

Epoch 400, Loss: 0.3968808352947235


Accuracy: 0.859
AUC: 0.902
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.611           0.050
Actual Positive       0.091           0.248

--------------------------just sybils-----------------------------

Accuracy: 0.731
Accuracy with optimal threshold: 0.8589091971290269
Accuracy with optimal threshold just for sybils: 0.7313414469069565


In [202]:
# Initialize model, optimizer, and loss function
input_dim = 1
hidden_dim = 4
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GAT(input_dim, hidden_dim, output_dim, heads=4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in tqdm(range(400)):
    optimizer.zero_grad()
    out = model(train_x, train_edge_index)
    loss = loss_fn(out, train_y)
    loss.backward()
    optimizer.step()
    if (epoch+1)%20 == 0 or epoch==0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute metrics
    acc = accuracy_score(test_y, pred)
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')  # Adjust depending on binary or multi-class
    cm = confusion_matrix(test_y, pred)
    cm = cm.astype('float') / len(pred)


    print(f'Accuracy: {acc:.3f}')
    print(f'AUC: {auc:.3f}')
    print('Confusion Matrix:')
    print(f"                  Predicted Negative    Predicted Positive")
    print(f"Actual Negative       {cm[0, 0]:.3f}           {cm[0, 1]:.3f}")
    print(f"Actual Positive       {cm[1, 0]:.3f}           {cm[1, 1]:.3f}")


    print("\n--------------------------just sybils-----------------------------\n")

    test_out = model(test_x, test_edge_index)
    test_out = test_out[mask2]
    pred = test_out.argmax(dim=1).cpu()
    acc = accuracy_score(test_y2, pred)
    print(f'Accuracy: {acc:.3f}')

# model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute AUC score as usual
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')
    fpr, tpr, thresholds = roc_curve(test_y, test_out[:, 1])
    j_scores = tpr - fpr
    optimal_threshold = thresholds[j_scores.argmax()]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc = accuracy_score(test_y, pred)

    # just sybil
    test_out = test_out[mask2]
    pred = (test_out[:, 1] >= optimal_threshold)
    acc_sybils = accuracy_score(test_y2, pred)
    print("Accuracy with optimal threshold:", acc)
    print("Accuracy with optimal threshold just for sybils:", acc_sybils)

  0%|          | 1/400 [00:00<01:25,  4.67it/s]

Epoch 1, Loss: 0.676198422908783


  5%|▌         | 20/400 [00:04<01:23,  4.54it/s]

Epoch 20, Loss: 0.6228294968605042


 10%|█         | 40/400 [00:08<01:30,  3.97it/s]

...

 95%|█████████▌| 380/400 [01:33<00:04,  4.53it/s]

Epoch 380, Loss: 0.3998268246650696


100%|██████████| 400/400 [01:38<00:00,  4.04it/s]

Epoch 400, Loss: 0.3997393548488617


Accuracy: 0.802
AUC: 0.902
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.532           0.129
Actual Positive       0.069           0.270

--------------------------just sybils-----------------------------

Accuracy: 0.796
Accuracy with optimal threshold: 0.8590302119846436
Accuracy with optimal threshold just for sybils: 0.7281542645484415
